In [13]:
from typing import TypedDict,List,Annotated
from langgraph.graph import END, START, StateGraph
import operator
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

In [14]:
class SimpleState(TypedDict):
    # count: int
    count: Annotated[int,operator.add]
    # summation: int
    summation: Annotated[int,operator.add]
    # tracking_sums: List['names']
    tracking_sums: Annotated[List[int],operator.concat]

In [ ]:
def increment(state:SimpleState) -> SimpleState:
    # new_count=state['count'] + 1
    # new_sum=state['summation'] + new_count

    state['count']=1
    state['summation']=state['count']+1
    state['tracking_sums']=[state['count']+1]
    print(state)
    return state
    return {
        # "count": new_count,
        "count":1,
        # "summation": new_sum,
        "summation": state['count'] + 1,
        # "tracking_sums":state['tracking_sums'] +[new_sum],
        "tracking_sums": [state['count'] + 1]
    }

In [28]:
def should_continue(state:SimpleState):
    if state['count']<5:
        return "continue"
    else:
        return "stop"

In [32]:
graph=StateGraph(state_schema=SimpleState)

In [33]:
graph.add_node(node="increment",action=increment)

In [34]:
graph.set_entry_point(key="increment")

In [35]:
graph.add_conditional_edges(
                 source="increment",
                 path=should_continue,
                 path_map={
                     "continue": "increment",
                     "stop": END
                 }
             )

In [36]:
app=graph.compile()

In [37]:
display(Image(data=app.get_graph(xray=True).draw_mermaid_png(max_retries=3)))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 3 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [38]:
state={"count":0,"summation":0,"tracking_sums":[]}

In [39]:
result=app.invoke(input=state)

In [40]:
result

{'count': 5, 'summation': 10, 'tracking_sums': [2, 2, 2, 2, 2]}